In [378]:
import pandas as pd
import numpy as np

In [379]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

In [380]:
print(train.shape)
train.head(1)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [381]:
print(test.shape)
test.head(1)

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [382]:
print("train null count")
print(train.isnull().sum())
print("----------------")
print("test null count")
print(test.isnull().sum())

train null count
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------
test null count
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [383]:
def setEmbarked(val):
    result = None
    if val == "Q": result = 2
    elif val == "C": result = 1
    else: result = 0
    return result

In [384]:
train["Embarked"] = train["Embarked"].apply(setEmbarked)
test["Embarked"] = test["Embarked"].apply(setEmbarked)

In [385]:
def setSex(val):
    result = None
    if val == "male": result = 1
    else: result = 0
    return result

In [386]:
train["Sex"] = train["Sex"].apply(setSex)
test["Sex"] = test["Sex"].apply(setSex)

In [387]:
def setCabin(val):
    result = 0
    if pd.isna(val): result = 0
    else: result = 1
    return result

In [388]:
train["HasCabin"] = train["Cabin"].apply(setCabin)
test["HasCabin"] = test["Cabin"].apply(setCabin)
train = train.drop(columns=["Cabin"], axis=1)
test = test.drop(columns="Cabin", axis=1)

In [389]:
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

In [390]:
train["FamilySize"] = train["SibSp"] + train["Parch"]
test["FamilySize"] = test["SibSp"] + test["Parch"]
train = train.drop(columns=["SibSp", "Parch"], axis=1)
test = test.drop(columns=["SibSp", "Parch"], axis=1)

In [391]:
train["Title"] = train["Name"].str.extract(', ([A-Z][a-z]+)')
test["Title"] = test["Name"].str.extract(', ([A-Z][a-z]+)')

In [392]:
def setTitle(val):
    result = 0
    if val == "Mr": result = 0
    elif val == "Miss": result = 1
    elif val == "Mrs": result = 2
    elif val == "Master": result = 3
    else: val = 4
    return result

In [393]:
train["Title"] = train["Title"].apply(setTitle)
test["Title"] = test["Title"].apply(setTitle)

In [394]:
train = train.drop(columns=["Ticket", "Name"], axis=1)
test = test.drop(columns=["Ticket", "Name"], axis=1)

In [395]:
train["FareBand"] = pd.cut(train["Fare"], 4)
train[["FareBand", "Survived"]].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.512, 128.082]",0.368113
1,"(128.082, 256.165]",0.724138
2,"(256.165, 384.247]",0.666667
3,"(384.247, 512.329]",1.000000


In [396]:
def setFare(val):
    result = 0
    if val < 128.082: result = 0
    elif val < 256.165: result = 1
    elif val < 384.247: result = 2
    else: result = 3
    return result

In [397]:
train["Fare"] = train["Fare"].apply(setFare)
test["Fare"] = test["Fare"].apply(setFare)

In [398]:
def setAlone(val):
    if val == 0 : return 1
    else: return 0

In [399]:
train["IsAlone"] = train["FamilySize"].apply(setAlone)
test["IsAlone"] = test["FamilySize"].apply(setAlone)

In [400]:
train["FamilyBand"] = pd.cut(train["FamilySize"], 4)
train[["FamilyBand", "Survived"]].groupby(['FamilyBand'], as_index=False).mean().sort_values(by='FamilyBand', ascending=True)

,FamilyBand,Survived
0,"(-0.01, 2.5]",0.388750
1,"(2.5, 5.0]",0.409091
2,"(5.0, 7.5]",0.222222
3,"(7.5, 10.0]",0.000000


In [401]:
def setFamilyBand(val):
    result = 0
    if val<2.5: result = 0
    elif val<5.0: result = 1
    elif val<7.5: result = 2
    else: result = 3
    return result

In [402]:
train["FamilySize"] = train["FamilySize"].apply(setFamilyBand)
test["FamilySize"] = test["FamilySize"].apply(setFamilyBand)

In [403]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,IsAlone
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.006345,-0.030467,0.019919,-0.027133,-0.042770,0.057462
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,0.147466,0.106811,0.316912,-0.077962,0.474182,-0.203367
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,-0.298580,0.045702,-0.725541,0.129124,-0.059659,0.135207
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114771,-0.116569,-0.140391,-0.111924,-0.629413,0.303646
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,0.036128,0.010171,0.249732,-0.274024,-0.268101,0.198270
Fare,-0.006345,0.147466,-0.298580,-0.114771,0.036128,1.000000,0.047596,0.259115,0.108431,0.056025,-0.094555
Embarked,-0.030467,0.106811,0.045702,-0.116569,0.010171,0.047596,1.000000,0.013774,-0.079096,0.034902,0.017807
HasCabin,0.019919,0.316912,-0.725541,-0.140391,0.249732,0.259115,0.013774,1.000000,-0.072565,0.089252,-0.158029
FamilySize,-0.027133,-0.077962,0.129124,-0.111924,-0.274024,0.108431,-0.079096,-0.072565,1.000000,0.312923,-0.383488
Title,-0.042770,0.474182,-0.059659,-0.629413,-0.268101,0.056025,0.034902,0.089252,0.312923,1.000000,-0.498594


In [404]:
train_age_yes = train[train["Age"].notna()]
train_age_no = train[train["Age"].isna()]

In [405]:
import statsmodels.api as sm
x_table = train_age_yes.drop(columns=["PassengerId", "Survived", "FareBand", "FamilyBand", "Age"], axis=1)
y_table = train_age_yes["Age"]
mod = sm.OLS(y_table,x_table)
fii = mod.fit()

display(fii.summary())
p_values = fii.summary2().tables[1]['P>|t|']
display(p_values)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    Age   R-squared (uncentered):                   0.789
Model:                            OLS   Adj. R-squared (uncentered):              0.787
Method:                 Least Squares   F-statistic:                              330.9
Date:                Mon, 23 Mar 2020   Prob (F-statistic):                   5.17e-233
Time:                        23:43:17   Log-Likelihood:                         -2954.7
No. Observations:                 714   AIC:                                      5925.
Df Residuals:                     706   BIC:                                      5962.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass         5.3388      0.633      8.439      0.000       4.097       6.581
Sex            7.8358      1.420      5.518      0.000       5.048      10.624
Fare           2.9368      1.888      1.555      0.120      -0.771       6.644
Embarked       1.3050      1.106      1.180      0.238      -0.866       3.476
HasCabin      19.2648      1.431     13.467      0.000      16.456      22.073
FamilySize    -4.2657      1.319     -3.233      0.001      -6.856      -1.675
Title          2.0807      0.825      2.522      0.012       0.461       3.700
IsAlone        8.9542      1.337      6.696      0.000       6.329      11.580
==============================================================================
Omnibus:                       15.211   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               15.684
Skew:                           0.361   Prob(JB):                     0.000393
Kurtosis:                       3.074   Cond. No.                         9.28
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pclass        1.810613e-16
Sex           4.827754e-08
Fare          1.203306e-01
Embarked      2.384230e-01
HasCabin      5.875518e-37
FamilySize    1.280667e-03
Title         1.187415e-02
IsAlone       4.375219e-11
Name: P>|t|, dtype: float64

In [406]:
x = train_age_yes[["Pclass", "FamilySize"]]
y = train_age_yes["Age"]

In [407]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
score = make_scorer(mean_squared_error)
k_fold = KFold(n_splits=600, shuffle=True, random_state=42)

In [408]:
lr = LinearRegression()
lr.fit(x, y)
scoring = "mean_squared_error"
score = cross_val_score(lr, x, y, cv=k_fold, n_jobs=1, scoring=score)
print(np.sqrt(score))
print("LinearRegression: {0: .4f}".format(round(np.mean(np.sqrt(score)))))

[21.78684646 10.2422246  20.79355505 22.42694128  2.6317261  14.95427169
  9.68464873  2.82969535 15.7890456  11.8934279  12.79542043 12.46862372
  7.29819174  8.57571506 11.75019603  8.49163581  6.32844706 10.25320333
 12.81775625 23.90752986 13.09625759 15.80152111 10.15129891  1.5929138
  7.21203272  5.84827315  7.94164167 17.59809688 10.16269934 21.11755688
 14.1998881  11.21348837 11.274403   18.3664653   4.89127355 18.25267884
  8.60102456 13.96115582 11.05750219 14.88104692  5.21819622 10.9770256
  4.58160452 10.47017243  9.48087873  4.93458464 12.76493862  1.30448147
 11.9221077  18.80568707  9.80692925 12.14076425 15.37437986  4.71009529
 13.71903405  3.94858355 10.33370559  7.09081692  9.59856822 11.99296576
 15.02240861  4.58160452  4.58342636 11.40570696  4.63150245  1.51119003
  1.30448147 12.95283088  4.79799512  9.05965958 10.02133541 13.93685095
  2.61982252  6.94511454  3.36062353  7.71130472  6.66111481  7.40644643
  3.64293676 20.08082441  2.02532922  2.14886574 10.1

In [409]:
train_age_yes.head(1)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,FareBand,IsAlone,FamilyBand
0,1,0,3,1,22.0,0,0,0,0,0,"(-0.512, 128.082]",0,"(-0.01, 2.5]"


In [410]:
y_predict = lr.predict(train_age_no[["Pclass", "FamilySize"]])
train_age_no["Age"] = y_predict
train = pd.concat([train_age_yes, train_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [411]:
test_age_yes = test[test["Age"].notna()]
test_age_no = test[test["Age"].isna()]

In [412]:
x_test = test_age_no[["Pclass", "FamilySize"]]
y_predict = lr.predict(x_test)
test_age_no["Age"] = y_predict
test = pd.concat([test_age_yes, test_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [413]:
train["Age"].astype("int")
test["Age"].astype("int")

0      34
1      47
2      62
3      27
4      22
       ..
408    26
410    26
413    26
416    26
417    26
Name: Age, Length: 418, dtype: int64

In [414]:
train["AgeBand"] = pd.cut(train["Age"], 5)
train[["AgeBand", "Survived"]].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.514019
1,"(16.336, 32.252]",0.341564
2,"(32.252, 48.168]",0.412844
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [415]:
def setAge(val):
    if val < 16.336: result = 0
    elif val < 32.252: result = 1
    elif val < 48.168: result = 2
    elif val <  64.084: result = 3
    else: result = 4
    return result

In [416]:
train["Age"] = train["Age"].apply(setAge)
test["Age"] = test["Age"].apply(setAge)

In [417]:
train["TitleAge"] = train["Age"] * train["Title"]
test["TitleAge"] = test["Age"] * test["Title"]

In [418]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,IsAlone,TitleAge
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.021435,-0.006345,-0.030467,0.019919,-0.027133,-0.042770,0.057462,-0.004571
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.028550,0.147466,0.106811,0.316912,-0.077962,0.474182,-0.203367,0.429170
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.423480,-0.298580,0.045702,-0.725541,0.129124,-0.059659,0.135207,-0.236529
Sex,0.042939,-0.543351,0.131900,1.000000,0.074858,-0.114771,-0.116569,-0.140391,-0.111924,-0.629413,0.303646,-0.672112
Age,0.021435,-0.028550,-0.423480,0.074858,1.000000,0.074494,-0.044622,0.291775,-0.277238,-0.201429,0.146935,0.315026
Fare,-0.006345,0.147466,-0.298580,-0.114771,0.074494,1.000000,0.047596,0.259115,0.108431,0.056025,-0.094555,0.100328
Embarked,-0.030467,0.106811,0.045702,-0.116569,-0.044622,0.047596,1.000000,0.013774,-0.079096,0.034902,0.017807,0.013392
HasCabin,0.019919,0.316912,-0.725541,-0.140391,0.291775,0.259115,0.013774,1.000000,-0.072565,0.089252,-0.158029,0.198546
FamilySize,-0.027133,-0.077962,0.129124,-0.111924,-0.277238,0.108431,-0.079096,-0.072565,1.000000,0.312923,-0.383488,0.016603
Title,-0.042770,0.474182,-0.059659,-0.629413,-0.201429,0.056025,0.034902,0.089252,0.312923,1.000000,-0.498594,0.639538


In [419]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,FareBand,IsAlone,FamilyBand,AgeBand,TitleAge
0,1,0,3,1,1,0,0,0,0,0,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(16.336, 32.252]",0
1,2,1,1,0,2,0,1,1,0,2,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(32.252, 48.168]",4
2,3,1,3,0,1,0,0,0,0,1,"(-0.512, 128.082]",1,"(-0.01, 2.5]","(16.336, 32.252]",1
3,4,1,1,0,2,0,0,1,0,2,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(32.252, 48.168]",4
4,5,0,3,1,2,0,0,0,0,0,"(-0.512, 128.082]",1,"(-0.01, 2.5]","(32.252, 48.168]",0


In [420]:
# x = train.drop(columns=["PassengerId", "Survived", "FareBand", "AgeBand", "FamilyBand", "Age", "FamilySize"], axis=1)
x = train.drop(columns=["PassengerId", "Survived", "FareBand", "AgeBand", "FamilyBand"], axis=1)
y = train["Survived"]

In [421]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=200, shuffle=True, random_state=42)

scoring = 'accuracy'

In [422]:
de = DecisionTreeClassifier()
de.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(de, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("DecisionTree: {0: .4f}".format(round(np.mean(score)*100, 2)))

[0.8  0.8  0.8  1.   0.6  0.4  1.   1.   0.8  0.4  1.   1.   1.   0.6
 0.8  1.   1.   0.8  0.8  0.8  0.8  0.8  0.6  1.   0.8  0.8  0.6  1.
 1.   1.   0.8  0.6  1.   0.6  1.   0.8  1.   0.8  1.   0.6  0.8  1.
 0.8  0.6  1.   0.4  0.4  0.6  0.8  1.   0.6  0.6  1.   0.8  0.4  0.8
 1.   1.   0.4  0.6  0.6  1.   0.8  1.   0.8  0.6  0.6  0.8  0.6  0.6
 1.   0.8  0.8  0.6  0.6  0.8  0.8  1.   0.8  1.   0.6  1.   0.6  0.6
 1.   0.8  1.   0.8  0.6  0.8  1.   1.   1.   0.75 0.75 0.75 1.   0.75
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 1.   1.   0.75
 0.75 0.5  0.75 0.75 1.   0.75 1.   1.   0.75 0.75 0.75 1.   0.75 0.75
 0.75 0.75 0.75 1.   0.75 0.75 0.75 1.   1.   1.   0.75 1.   0.75 1.
 0.75 1.   0.75 0.75 1.   0.75 1.   0.5  1.   1.   1.   1.   0.75 0.75
 0.75 0.75 0.75 1.   0.75 1.   0.75 0.75 0.75 1.   0.75 0.75 0.25 0.5
 0.75 1.   0.5  0.75 0.5  1.   1.   1.   1.   1.   1.   0.25 1.   1.
 0.75 0.75 0.75 1.   0.75 0.75 0.5  1.   0.75 0.5  1.   0.5  0.75 0.75
 0.75 1.   0.5  1. 

In [423]:
sv = SVC()
sv.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(sv, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("SupportVectorMachine: {0: .4f}".format(round(np.mean(score)*100, 2)))

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

[0.8  0.8  0.8  1.   0.6  0.8  1.   0.8  0.8  0.4  1.   1.   1.   0.6
 0.8  1.   1.   0.8  0.8  0.8  0.8  1.   0.6  1.   0.6  0.8  0.6  1.
 0.8  0.8  1.   0.6  1.   0.8  1.   0.8  1.   0.6  1.   0.8  0.8  1.
 0.8  0.6  1.   1.   0.4  1.   1.   1.   0.8  0.8  1.   0.8  0.4  0.8
 1.   1.   0.6  0.6  0.8  1.   0.8  0.8  0.8  0.8  0.8  1.   0.6  0.8
 1.   0.8  0.8  0.4  0.8  0.8  0.6  1.   0.6  1.   0.6  0.8  0.6  0.8
 1.   0.8  1.   0.8  0.6  0.6  1.   1.   1.   0.75 0.5  1.   1.   0.75
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 0.75 1.   0.75
 0.75 0.5  0.75 0.75 1.   0.75 1.   1.   0.5  0.75 0.75 0.75 1.   0.75
 1.   1.   1.   1.   1.   0.75 0.75 1.   1.   1.   0.75 1.   0.5  1.
 0.5  1.   1.   0.75 1.   0.5  1.   0.75 1.   1.   1.   1.   0.75 0.75
 1.   1.   0.75 1.   0.75 1.   0.75 1.   1.   1.   0.75 1.   0.5  0.75
 0.5  1.   0.5  1.   1.   1.   0.75 0.75 1.   1.   1.   0.25 1.   0.75
 1.   0.75 0.75 1.   0.75 0.75 0.75 1.   0.75 0.75 1.   0.75 0.75 1.
 0.75 1.   0.75 1.

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/skyepodium/opt/anacon

In [424]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(xgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("xgboost: {0: .4f}".format(round(np.mean(score)*100, 2)))

[0.8  0.6  0.8  1.   0.8  0.8  1.   1.   0.8  0.4  1.   1.   1.   0.6
 0.8  1.   1.   0.8  0.8  1.   0.8  0.8  0.6  1.   0.8  0.8  0.6  1.
 1.   1.   1.   0.6  1.   0.6  1.   0.8  1.   0.8  1.   0.6  0.8  1.
 0.8  0.6  1.   0.8  0.4  0.8  1.   1.   0.6  0.8  1.   1.   0.4  0.8
 1.   1.   0.4  0.6  0.6  1.   0.8  1.   0.8  0.6  0.8  0.8  0.6  0.4
 1.   0.8  0.8  0.4  0.8  0.8  0.6  1.   0.8  1.   0.6  1.   0.8  0.6
 1.   0.8  1.   0.8  0.6  0.8  1.   1.   1.   0.75 0.5  1.   1.   1.
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 0.75 1.   0.75
 0.75 0.5  0.75 0.75 1.   0.75 1.   1.   0.75 0.75 0.75 0.75 1.   0.75
 0.75 0.75 1.   1.   1.   0.75 0.75 1.   1.   1.   0.75 1.   0.75 0.75
 0.75 1.   0.75 0.75 1.   0.75 1.   0.5  1.   1.   1.   1.   0.75 0.75
 1.   1.   0.75 1.   0.75 1.   0.75 1.   1.   1.   0.75 0.75 0.5  0.75
 0.75 1.   0.5  1.   1.   1.   0.75 1.   1.   1.   1.   0.25 1.   1.
 1.   0.75 0.75 1.   0.75 1.   0.5  1.   0.75 0.5  1.   0.5  0.75 1.
 0.75 1.   0.75 1.  

In [425]:
lgb_model = lgb.LGBMClassifier(objective="binary", random_state=42)
lgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(lgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("lightgbm: {0: .4f}".format(round(np.mean(score)*100, 2)))

[0.8  0.6  0.8  1.   0.8  0.8  1.   1.   0.8  0.6  1.   1.   1.   0.6
 0.8  1.   1.   0.8  0.8  1.   0.8  0.8  0.6  1.   0.8  0.8  0.6  1.
 1.   1.   1.   0.6  1.   0.6  1.   0.8  1.   0.8  1.   0.6  0.8  1.
 0.8  0.6  1.   0.8  0.4  0.8  1.   1.   0.6  0.8  1.   1.   0.4  0.8
 1.   1.   0.4  0.6  0.6  1.   0.8  1.   0.8  0.6  0.6  0.8  0.6  0.4
 1.   0.8  0.8  0.4  0.8  0.8  0.6  1.   0.8  1.   0.6  1.   0.8  0.6
 1.   0.6  1.   0.8  0.6  0.8  1.   1.   1.   0.75 0.75 1.   1.   0.75
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 1.   1.   0.75
 0.75 0.5  0.5  0.75 1.   0.75 1.   1.   0.75 0.75 0.75 1.   0.75 0.75
 1.   0.75 1.   1.   1.   0.75 0.75 1.   1.   1.   0.75 1.   0.75 0.75
 0.75 1.   0.75 0.75 1.   0.75 1.   0.5  1.   1.   1.   1.   0.75 0.75
 0.75 1.   0.75 1.   0.75 1.   0.75 1.   0.75 1.   0.75 0.75 0.5  0.75
 0.75 1.   0.5  1.   1.   1.   0.75 1.   1.   1.   1.   0.25 1.   1.
 1.   0.75 0.75 1.   0.75 1.   0.5  1.   0.75 0.5  1.   0.5  0.75 1.
 0.75 1.   0.5  1.

In [426]:
x_test = test.drop(columns=["PassengerId"], axis=1)

In [427]:
predict = lgb_model.predict(x_test)

In [428]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predict
})

In [429]:
submission.to_csv("0323_LightGBM_{0: .4f}.csv".format(82.65), index=False)